<a href="https://colab.research.google.com/github/laharimarina/FMML_ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1)
The percentage of data allocated to the validation set in a machine learning experiment can have a significant impact on the accuracy and overall performance of your model. Let's discuss how changes in the percentage of the validation set affect your results:

Increase in Validation Set Percentage:

Pros:

A larger validation set means more data for evaluating the model's performance. This can provide a more reliable estimate of how well your model generalizes to unseen data.
It helps reduce the risk of overfitting because the model has less training data to memorize, and the validation set becomes a better representation of unseen data.
Cons:

With a larger validation set, you have less data for training your model. This can lead to underfitting if the training set is too small or not representative of the underlying data distribution.
Effect on Validation Accuracy:

Initially, increasing the validation set percentage may lead to a more accurate estimate of the model's performance on unseen data. However, if the validation set becomes too large, it can lead to underfitting, and the validation accuracy may start to drop.
Decrease in Validation Set Percentage:

Pros:

A smaller validation set leaves more data available for training, which can help the model learn more complex patterns and potentially improve its performance.
Cons:

A smaller validation set may lead to an inaccurate estimate of model performance, as it may be more sensitive to noise or outliers in the validation data.
There's a higher risk of overfitting because the model has more training data and may fit the training data too closely without a reliable validation set for monitoring.
Effect on Validation Accuracy:

Reducing the validation set percentage may result in a higher training accuracy, but it can lead to an overly optimistic estimate of the model's generalization performance. The model might perform poorly on new, unseen data.

2)
The size of the training and validation sets can significantly affect how well you can predict the accuracy of your model on the test set using the validation set. This relationship is particularly important in machine learning because it impacts the reliability of your model evaluation and generalization estimates. Here's how the sizes of these sets can influence predictions of test set accuracy:

Large Training Set and Small Validation Set:

When you have a large training set and a small validation set, the model has access to more data for learning patterns and generalizing from the training data.
The validation set may not be as representative of unseen data, potentially leading to overfitting. In this case, the model might perform well on the validation set but poorly on the test set, which reflects how well it generalizes to new, unseen data.
Small Training Set and Large Validation Set:

A small training set provides limited data for the model to learn from, potentially leading to underfitting.
With a large validation set, you have a better estimate of how well the model generalizes to unseen data. However, the model's performance on the validation set may not be as high due to the small training set, which could make you underestimate its true potential.
Balanced Training and Validation Sets:

Having a balanced allocation of data between the training and validation sets is often recommended to strike a good balance between model training and evaluation.
It provides a more accurate estimate of your model's generalization performance because both the training and validation sets are representative of the overall data distribution.
This balance helps you make more reliable predictions of the model's accuracy on the test set. If the model performs well on the validation set, it is more likely to perform well on the test set, assuming the validation set is a good representative sample.

3)80/20 Rule: A common starting point is to reserve 80% of your data for training and 20% for validation. This split can work well for medium-sized datasets.

70/30 Rule: If you have a small dataset, you might use a 70/30 or even a 60/40 split to allocate more data for training. This helps your model learn better, but you have less data for validation.

Cross-Validation: For small datasets or when you want to be more confident in your model's performance, you can use techniques like k-fold cross-validation. In k-fold cross-validation, the data is divided into 'k' subsets, and the model is trained and validated 'k' times, with each subset serving as the validation set once. This ensures that every data point is used for validation at least once.

Leave-One-Out Cross-Validation (LOOCV): If your dataset is extremely small, you can use LOOCV, where each data point is used as the validation set once while the rest is used for training. However, this can be computationally expensive.

Stratified Sampling: If your dataset is imbalanced (e.g., one class has significantly fewer samples than another), you may want to use stratified sampling to ensure that the validation set maintains the same class distribution as the training set.

Time-Series Data: For time-series data, you should reserve a chronological split for validation. For example, you might reserve the last 20% of your data for validation to simulate how your model will perform on future, unseen data.

Hyperparameter Tuning: When performing hyperparameter tuning, you might further split the validation set into a smaller validation set and a test set. For example, you might use 70% for training, 15% for validation, and 15% for testing.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1)Reduced Variance: A single validation split may be particularly unrepresentative of your data, leading to overly optimistic or pessimistic estimates of your model's performance. By averaging over multiple splits, you reduce the impact of such variability, giving you a more stable and representative estimate.

Better Generalization Assessment: Cross-validation helps you gauge how well your model generalizes to unseen data. It simulates the model's performance on different subsets of the data, providing a more robust evaluation of its ability to handle variations and noise in the data.

Utilizing All Data: In traditional train-test splits, you may end up excluding a substantial portion of your data for validation purposes. Cross-validation allows you to make better use of your entire dataset by cycling through different data subsets for validation.

Common cross-validation techniques include k-fold cross-validation and stratified k-fold cross-validation:

K-fold Cross-Validation: The dataset is divided into k subsets (folds). The model is trained and validated k times, each time using a different fold for validation and the remaining k-1 folds for training. The validation scores are then averaged.

Stratified K-fold Cross-Validation: This variation of k-fold cross-validation ensures that each fold has a similar distribution of target classes as the whole dataset. It is particularly useful when dealing with imbalanced datasets.

2)Reduced Overfitting: Cross-validation helps mitigate the risk of overfitting to a specific validation split. If you were to rely on a single validation split, there's a chance that the model might perform exceptionally well on that particular subset of data but poorly on unseen data. Cross-validation averages the performance over multiple splits, making it less likely for your model to be over-optimized for a single validation set.

Better Generalization Assessment: By assessing model performance on multiple subsets of the data, cross-validation provides a more comprehensive view of how well your model generalizes to different portions of the dataset. This helps you understand how robust your model is and how it might perform on unseen data, which is the primary goal when estimating test accuracy.

Improved Model Selection: Cross-validation is often used for comparing and selecting the best model among different algorithms or hyperparameter settings. Averaging the performance over multiple splits gives you a more reliable basis for model selection.

However, it's essential to remember that cross-validation is still an estimate of test accuracy, and the actual test accuracy on completely unseen data may differ. To obtain the most accurate estimate of test accuracy, you should reserve a separate, untouched test dataset that your model has never seen during training or cross-validation. This dataset serves as an independent benchmark to evaluate your model's performance in a real-world scenario.2)

3)Convergence: Many iterative algorithms aim to converge to a stable solution. Initially, as you increase the number of iterations, the estimate may improve and get closer to the true solution. However, there is a point of diminishing returns, where additional iterations may not significantly improve the estimate because the algorithm has already converged to a stable solution.

Computational Cost: More iterations typically require more computational resources (time and/or processing power). Therefore, there is a trade-off between the accuracy of the estimate and the computational cost. Sometimes, the marginal improvement in the estimate with additional iterations may not be worth the increased computational cost.

Convergence Rate: Some algorithms have a fast convergence rate, meaning they reach a good estimate relatively quickly. In such cases, increasing the number of iterations may not provide a significant benefit in terms of estimate quality.

Numerical Stability: Excessive iterations in certain algorithms can lead to numerical instability or loss of precision. This is especially true if the algorithm involves numerical approximations that accumulate errors over time.

Overfitting: In some machine learning and optimization problems, increasing the number of iterations too much can lead to overfitting, where the model becomes overly tailored to the training data and performs poorly on new, unseen data.

4)Overfitting: If you have a very small training dataset and you increase the number of iterations too much, the model might start memorizing the training data instead of learning meaningful patterns. This can lead to overfitting, where the model performs well on the training data but poorly on new, unseen data.

Diminishing Returns: There is a point of diminishing returns when it comes to increasing the number of iterations. Initially, more iterations may lead to better generalization, but beyond a certain point, the model's performance on the validation dataset may plateau, or it may even degrade due to overfitting.

Computational Cost: Increasing the number of iterations also increases the computational cost, which may not be feasible in some cases, especially when training large models on limited hardware resources.

To address the limitations of small datasets, consider the following strategies in addition to increasing iterations:

Data Augmentation: You can artificially increase the size of your training dataset by applying various data augmentation techniques, such as rotating, flipping, or cropping images, or adding noise to text data.

Transfer Learning: If you have access to pre-trained models on larger datasets, you can fine-tune these models on your small dataset. Transfer learning allows you to leverage the knowledge learned from the larger dataset and adapt it to your specific task.

Regularization: Use regularization techniques like dropout, L1/L2 regularization, or early stopping to prevent overfitting when increasing the number of iterations.

Cross-Validation: Implement cross-validation to better estimate your model's performance and reduce the risk of overfitting on a small dataset.